# Installation of the required dependencies

In [1]:
!apt update
!apt install -qq -y libmagic-dev poppler-utils tesseract-ocr

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
44 packages can be upgraded. Run 'apt list --upgradable' to see them.
tesseract-ocr is already the newest version (4.1.1-2.1build1).
libmagic-dev is already the newest 

In [2]:
!python -m pip install poetry==1.4.0

In [3]:
!python -m pip install git+https://github.com/dataforgoodfr/12_taxobservatory.git

  Cloning https://github.com/dataforgoodfr/12_taxobservatory.git to /tmp/pip-req-build-podjm5ar
  Running command git clone --filter=blob:none --quiet https://github.com/dataforgoodfr/12_taxobservatory.git /tmp/pip-req-build-podjm5ar
  Resolved https://github.com/dataforgoodfr/12_taxobservatory.git to commit ee7c66e7c4f4790f5349d2e9a070afcfc41766f4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
# Fix numpy issue https://github.com/numpy/numpy/issues/25150
!python -m pip install numpy==1.24.4

  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.2.0 which is incompatible.
tensorflow 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.
unstructured 0.12.6 requires numpy==1.26.4, but you have numpy 1.24.4 which is incompatible.


In [5]:
!python -m pip install huggingface_hub hf_transfer
%env HF_HUB_ENABLE_HF_TRANSFER=1

env: HF_HUB_ENABLE_HF_TRANSFER=1


# Getting a sample PDF

In [6]:
from huggingface_hub import hf_hub_download

file_path = hf_hub_download(repo_id="DataForGood/taxobservatory-pdfs", repo_type="dataset", filename="Acciona_2020_CbCR_1.pdf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Creation of the pipeline

## Definition of the configuration to use

In [7]:
pagefilter = {"type": "RFClassifier",
              "params": {
                  "modelfile": "random_forest_model_low_false_positive.joblib"
              }
             }

table_extraction = {
    "img": [
        {
            "type": "Camelot",
             "params": {
                 "flavor": "stream"
             }
        },
        {
            "type": "Unstructured",
            "params": {
                "pdf_image_dpi": 300,
                "hi_res_model": "yolox"
            }
        }
    ]
}
config= {"pagefilter": pagefilter, "table_extraction": table_extraction}

In [8]:
from country_by_country import processor

report_processor = processor.ReportProcessor(config)

In [9]:
report_processor.process(file_path)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'pagefilter': {'src_pdf': '/root/.cache/huggingface/hub/datasets--DataForGood--taxobservatory-pdfs/snapshots/98e726d3a02be4b8023ab1130a0af2e97aeaf9cf/Acciona_2020_CbCR_1.pdf', 'selected_pages': [0]}, 'text_table_extractors': {'camelot_stream': {'ntables': 4, 'tables': [                                                    0               1
0                                                      SUSTAINABILITY
1                                                              REPORT
2                                                                2020
3                              EXPONENTIAL LEADERSHIP                
4                                          Governance                
5                                                               Index
6                      Country-by-Country information                
7   The following table includes information on al...             1 L
8   Consolidated Financial Statements are resident...                
9                            

{'pagefilter': {'src_pdf': '/root/.cache/huggingface/hub/datasets--DataForGood--taxobservatory-pdfs/snapshots/98e726d3a02be4b8023ab1130a0af2e97aeaf9cf/Acciona_2020_CbCR_1.pdf',
  'selected_pages': [0]},
 'text_table_extractors': {'camelot_stream': {'ntables': 4,
   'tables': [                                                    0               1
    0                                                      SUSTAINABILITY
    1                                                              REPORT
    2                                                                2020
    3                              EXPONENTIAL LEADERSHIP                
    4                                          Governance                
    5                                                               Index
    6                      Country-by-Country information                
    7   The following table includes information on al...             1 L
    8   Consolidated Financial Statements are resident...    